In [9]:
import math
import numpy as np 
import pandas as pd
import cv2

In [7]:
def cur_price(h):
    p = 0.85
    k = 12.0 #эффективная проницаемость, мД (милиДарси)

    SRPLast = 270.0 #среднее пластовое давление, атм (атмосфер)
    zaboinoe_dav = 100.0 #забойное давление давление внизу скважины , атм
    vazkost = 1.4 #вязкость нефти, сП (сантиПуаз)
    obim_Koof = 1.15 #объёмный коэффициент нефти, д. ед.
    Rk = 800.0 #радиус контура объёма, из которого добывает скважина нефть, м ????????????
    Rc = 0.1 #радиус скважины, м
    SkinFack = 0.0 #скин − фактор (показывает ухудшение фильтрационных свойств пласта вблизи скважины)






    q = (p*k*h*(SRPLast - zaboinoe_dav))/(18.41*vazkost*obim_Koof*(math.log(Rk/Rc)-0.5+SkinFack))


    def qt(koofB,koofD):
        massiv = []

        for shagVR in range(480):
            QT = q/(1 + koofB * koofD * shagVR)**(1/koofB)
            massiv.append(QT*30)
        return massiv


    massiv=qt(1.4,0.12)
    price=[]
    price_tonna=53_135.006
    for n in range(40):
        try:
            summa=sum([massiv[i] for i in range(n*12,n*12+12)])
            cost=summa*price_tonna

            k=1/((1+0.17)**n)
            price.append(k*cost)
        except:
            break

    last_answer=sum(price)-60_000_000
    return last_answer

In [12]:
def f(row):
  if row['R']>140: return 0
  elif row['R']>100 : return 10
  elif row['R']>80 : return 20

  elif row['R']>35 : return 30

  elif row['R']>20 : return 40
  else: return 50

In [13]:
def matrix(photo):
    img = cv2.imread(photo)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    df = pd.DataFrame([list(l) for l in hsv]).stack().apply(pd.Series)
    df.index.name = 'st'
    df.columns = list('RGB')

    df['color']= df.apply (f, axis=1)

    data=[[_ for _ in range(926)] for _ in range(664)]
    for i in range(len(data)):
        for j in range(len(data[0])):
            data[i][j]=df.loc[i,j]['color']

    data_sector_j=[]
    for i in range(664):
        for j in range(926//32):
            data_sector_j+=[sum(data[i][j*32:j*32+32])//32]

    data_sector_i=[]
    for x in range(0,len(data_sector_j),28):
        data_sector_i+=[data_sector_j[x:x+28]]

    data_sector=[]
    co=0
    cash=np.array([0 for _ in range(28)])
    for x in data_sector_i:
        x = np.array(x)
        if co==32: 
            co=0
            data_sector+=[list(cash)]
            cash=np.array([0 for _ in range(28)]) 
        cash+=x
        co+=1

    for i in range(len(data_sector)):
        for j in range(len(data_sector[0])):
            data_sector[i][j]//=32
    return data_sector

In [14]:
data=matrix('new_test_photo.png')
data_mine=[[False for _ in range(len(data[0]))] for _ in range(len(data))]

In [19]:
answer=0
for i in range(len(data)):
    for j in range(len(data[0])):
        if data_mine[i][j]:
            answer+=cur_price(data[i][j])

In [20]:
answer

0

In [22]:
for _ in data:
    print(_)

[0, 0, 0, 0, 0, 0, 0, 5, 10, 10, 11, 12, 12, 12, 10, 10, 10, 10, 10, 10, 11, 12, 19, 20, 20, 20, 20, 20]
[0, 0, 0, 0, 0, 0, 2, 10, 11, 14, 18, 19, 20, 20, 19, 18, 14, 11, 10, 10, 10, 10, 17, 20, 20, 20, 20, 20]
[0, 0, 0, 0, 0, 0, 4, 10, 16, 20, 20, 21, 20, 20, 20, 20, 20, 18, 12, 10, 10, 10, 10, 16, 19, 20, 20, 20]
[0, 0, 0, 0, 0, 0, 6, 13, 19, 23, 28, 30, 29, 26, 21, 20, 20, 20, 17, 10, 10, 10, 10, 10, 12, 17, 19, 20]
[0, 0, 0, 0, 0, 1, 10, 18, 23, 30, 31, 33, 30, 30, 28, 20, 20, 20, 17, 10, 10, 10, 10, 10, 10, 11, 11, 15]
[0, 0, 0, 0, 0, 7, 14, 21, 29, 35, 40, 40, 38, 30, 30, 23, 20, 20, 15, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[0, 0, 0, 0, 5, 11, 18, 25, 32, 40, 40, 39, 40, 34, 30, 24, 20, 19, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[0, 0, 0, 4, 10, 15, 21, 29, 37, 41, 42, 42, 40, 36, 30, 23, 20, 17, 10, 10, 10, 10, 10, 10, 10, 10, 11, 10]
[0, 0, 5, 9, 12, 19, 25, 30, 39, 40, 50, 42, 40, 34, 30, 20, 19, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[0, 6, 10, 10, 17, 20, 29, 33, 40, 39